In [1]:
# 手写体识别数据集
# http://yann.lecun.com/exdb/mnist/
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import warnings
warnings.filterwarnings('ignore')
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

# batch
batch_size = 100
# 批次数
n_batch = mnist.train.num_examples // batch_size

# learning_rate
learning_rate = 0.2

# 占位符
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x_input')
    y = tf.placeholder(tf.float32,[None,10],name='y_input')
keep_probility = tf.placeholder(tf.float32)
lr = tf.Variable(0.001,dtype=tf.float32)

with tf.name_scope('Layer_1'):
# 创建一个单层的神经网络
    with tf.name_scope('Weight'):
        W1 = tf.Variable(tf.truncated_normal([784,100],stddev=0.1))
    with tf.name_scope('Bias'):
        b1 = tf.Variable(tf.zeros([100]) + 0.1)
    with tf.name_scope("Active"):
        L1 = tf.tanh(tf.matmul(x,W1) + b1)
    with tf.name_scope('DropOut'):
        L1_drop = tf.nn.dropout(L1,keep_prob=keep_probility)
with tf.name_scope('Layer_2'):
    W2 = tf.Variable(tf.truncated_normal([100,50],stddev=0.1))
    b2 = tf.Variable(tf.zeros([50]) + 0.1)
    L2 = tf.tanh(tf.matmul(L1_drop,W2) + b2)
    L2_drop = tf.nn.dropout(L2,keep_prob=keep_probility)
with tf.name_scope("Layer_3"):
    W3 = tf.Variable(tf.truncated_normal([50,30],stddev=0.1))
    b3 = tf.Variable(tf.zeros([30]) + 0.1)
    L3 = tf.tanh(tf.matmul(L2_drop,W3) + b3)
    L3_drop = tf.nn.dropout(L3,keep_prob=keep_probility)

with tf.name_scope("Output"):
    W4 = tf.Variable(tf.truncated_normal([30,10],stddev=0.1))
    b4 = tf.Variable(tf.zeros([10]) + 0.1)
    prediction = tf.nn.softmax(tf.matmul(L3_drop,W4) + b4)

# loss
# loss = tf.reduce_mean(tf.square(y - prediction))

# 使用交叉熵代价函数
with tf.name_scope("Loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

with tf.name_scope("Train"):
    # train
    # train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    train_step = tf.train.AdamOptimizer(lr).minimize(loss)
    
with tf.name_scope("Init"):
    # init
    init = tf.global_variables_initializer()
with tf.name_scope("Accuracy"):
    # accuracy
    correct_predictions = tf.equal(tf.arg_max(y,1),tf.arg_max(prediction,1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(21):
        sess.run(tf.assign(lr, 0.001 * (0.95 ** epoch)))
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_probility:0.9})
        lr_rate = sess.run(lr)
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_probility:0.9})
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_probility:0.9})
        print('epoch:'+ str(epoch) + ", Testing Accuracy:" + str(test_acc) + ", Training Accuracy:" + str(train_acc) + ", Learning Rate:" + str(lr_rate)) 

/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:461: FutureWarning: Passing (type, 1) or 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
epoch:0, Testing Accuracy:0.9295, Training Accuracy:0.9261636, Learning Rate:0.001
epoch:1, Testing Accuracy:0.9356, Training Accuracy:0.9393091, Learning Rate:0.00095
epoch:2, Testing Accuracy:0.9505, Training Accuracy:0.95225453, Learning Rate:0.0009025
epoch:3, Testing Accuracy:0.9503, Training Accuracy:0.95565456, Learning Rate:0.000857375
epoch:4, Testing Accuracy:0.9544, Training Accuracy:0.9617091, Learning Rate:0.00081450626
epoch:5, Testing Accuracy:0.9556, Training Accuracy:0.96458185, Learning Rate:0.0007737809
epoch:6, Testing Accuracy:0.9574, Training Accuracy:0.96672726, Learning Rate:0.0007350919
epoch:7, Testing Accuracy:0.9595, Training Accuracy:0.9680182, Learning Rate:0.0006983373
epoch:8, Testing Accuracy:0.9627, Training Accuracy:0.9697273, Learning Rate:0.0006634204
epoch:9